In [1]:
#將爬蟲出來的句子做前處理，像是刪除停用詞等等
import numpy as np
import pandas as pd

In [2]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank
#https://www.kaggle.com/cpmpml/spell-checker-using-word2vec

In [9]:
#spellchecker
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [10]:
from nltk.corpus import stopwords
#大眾形容詞加入停用詞
adj_list=["good","great","best","nice","bad","worse","worst","well","cool","restaurant","AM","pm","PM","true","false"]
#大寫的停用詞加入
stop_words_list=stopwords.words('english')+adj_list
for x in range(len(stop_words_list)):
    stop_words_list[x]=stop_words_list[x].title()
#全部的停用詞
stop_words = set(stopwords.words('english')+stop_words_list+adj_list)

In [19]:
import re
import nltk
from nltk.tokenize import word_tokenize

def preprocess(string):
    #去除英文以外
    string = re.sub(r'[^A-Za-z]+', ' ', string)
    #去除前後空格
    return string
def stopword_removal_append(string):    
    words = string.split()
    temp=0
    for r in words: 
        if not r in stop_words:
            r = correction(r)
            with open('./review_sets/review'+str(count)+'.txt','a', encoding = 'UTF8') as f1 : 
                f1.write(r+" ") 
                f1.close()
                temp=1
    if temp==0:
        with open('./review_sets/review'+str(count)+'.txt','a', encoding = 'UTF8') as f1 :
            f1.write("others")
            f1.close()

In [20]:
count=1
reviews = 995 #要隨著reviews筆數更改
for x in range(1,reviews+1): #總共1~reviews+1 
    df = pd.read_csv('original_review_sets/review'+str(x)+'.txt', sep='@', header=None,quoting=3)#quoting 防止有"會EOF
    string = df.values.tolist()[0][0]
    string = preprocess(string)
    stopword_removal_append(string)
    count+=1
#遇到會錯誤的那筆資料，請將original_review的那筆資料改為others